# Start

In [285]:
import pandas as pd
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

In [286]:
engine = create_engine("sqlite:///sqlite_db_pythonsqlite.db")
con = engine.connect()

In [287]:
print(engine.table_names())

['Bookings', 'Facilities', 'Members']


In [288]:
rs = con.execute("SELECT * FROM Members")
members = pd.DataFrame(rs.fetchall())
rs = con.execute("SELECT * FROM Bookings")
bookings = pd.DataFrame(rs.fetchall())
rs = con.execute("SELECT * FROM Facilities")
facilities = pd.DataFrame(rs.fetchall())

In [289]:
members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [290]:
bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [291]:
facilities

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


# Questions

## Q8

In [292]:
query = """SELECT m.memid, surname || ' ' || firstname AS full_name, starttime, name, slots,
CASE WHEN m.memid = 0 THEN guestcost ELSE membercost END AS cost,
CASE WHEN m.memid = 0 THEN 'guest' ELSE 'member' END AS guest_or_member,
CASE WHEN m.memid = 0 THEN guestcost*slots ELSE membercost*slots END AS fee
FROM Bookings b JOIN Members m ON b.memid = m.memid
JOIN Facilities f ON b.facid = f.facid
WHERE (starttime LIKE '%2012-09-14%') AND (CASE WHEN m.memid = 0 THEN guestcost*slots ELSE membercost*slots END > 30)
ORDER BY fee DESC;"""

rs = con.execute(query)
pd.DataFrame(rs.fetchall())

,memid,full_name,starttime,name,slots,cost,guest_or_member,fee
0,0,GUEST GUEST,2012-09-14 11:00:00,Massage Room 2,4,80.0,guest,320.0
1,0,GUEST GUEST,2012-09-14 09:00:00,Massage Room 1,2,80.0,guest,160.0
2,0,GUEST GUEST,2012-09-14 13:00:00,Massage Room 1,2,80.0,guest,160.0
3,0,GUEST GUEST,2012-09-14 16:00:00,Massage Room 1,2,80.0,guest,160.0
4,0,GUEST GUEST,2012-09-14 17:00:00,Tennis Court 2,6,25.0,guest,150.0
5,0,GUEST GUEST,2012-09-14 16:00:00,Tennis Court 1,3,25.0,guest,75.0
6,0,GUEST GUEST,2012-09-14 19:00:00,Tennis Court 1,3,25.0,guest,75.0
7,0,GUEST GUEST,2012-09-14 14:00:00,Tennis Court 2,3,25.0,guest,75.0
8,0,GUEST GUEST,2012-09-14 09:30:00,Squash Court,4,17.5,guest,70.0
9,13,Farrell Jemima,2012-09-14 14:00:00,Massage Room 1,4,9.9,member,39.6


## Q9

In [293]:
query = """SELECT memid, surname || ' ' || firstname AS full_name, starttime, name, slots,
CASE WHEN memid = 0 THEN guestcost ELSE membercost END AS cost,
CASE WHEN memid = 0 THEN 'guest' ELSE 'member' END AS guest_or_member,
CASE WHEN memid = 0 THEN guestcost*slots ELSE membercost*slots END AS fee

FROM
(SELECT bookid,m.memid,starttime,slots,surname,firstname, address, zipcode, telephone,recommendedby,
joindate,f.facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
FROM Bookings b JOIN Members m ON b.memid = m.memid JOIN Facilities f ON b.facid = f.facid
WHERE (starttime LIKE '%2012-09-14%') AND (CASE WHEN m.memid = 0 THEN guestcost*slots ELSE membercost*slots END > 30))

ORDER BY fee DESC;
"""

rs = con.execute(query)
pd.DataFrame(rs.fetchall())

,memid,full_name,starttime,name,slots,cost,guest_or_member,fee
0,0,GUEST GUEST,2012-09-14 11:00:00,Massage Room 2,4,80.0,guest,320.0
1,0,GUEST GUEST,2012-09-14 09:00:00,Massage Room 1,2,80.0,guest,160.0
2,0,GUEST GUEST,2012-09-14 13:00:00,Massage Room 1,2,80.0,guest,160.0
3,0,GUEST GUEST,2012-09-14 16:00:00,Massage Room 1,2,80.0,guest,160.0
4,0,GUEST GUEST,2012-09-14 17:00:00,Tennis Court 2,6,25.0,guest,150.0
5,0,GUEST GUEST,2012-09-14 16:00:00,Tennis Court 1,3,25.0,guest,75.0
6,0,GUEST GUEST,2012-09-14 19:00:00,Tennis Court 1,3,25.0,guest,75.0
7,0,GUEST GUEST,2012-09-14 14:00:00,Tennis Court 2,3,25.0,guest,75.0
8,0,GUEST GUEST,2012-09-14 09:30:00,Squash Court,4,17.5,guest,70.0
9,13,Farrell Jemima,2012-09-14 14:00:00,Massage Room 1,4,9.9,member,39.6


## Q10

In [301]:
rs = con.execute('''SELECT * FROM

(SELECT Facilities.name facility, SUM((memid<>0)*membercost + (memid=0)*guestcost) AS cost
FROM Bookings JOIN Facilities ON Bookings.facid = Facilities.facid GROUP BY facility)

WHERE cost > 1000 ORDER BY cost DESC''')

pd.DataFrame(rs.fetchall())

,facility,cost
0,Massage Room 1,20807.9
1,Massage Room 2,6987.3
2,Squash Court,4970.0
3,Tennis Court 2,4205.0
4,Tennis Court 1,4040.0


## Q11

In [295]:
rs = con.execute('''SELECT m1.memid,m1.firstname,m1.surname,m2.memid,m2.firstname,m2.surname
FROM Members m1 JOIN Members m2 ON m1.recommendedby = m2.memid
WHERE m1.recommendedby <> '' ORDER BY m1.firstname, m1.surname''')

for i in rs.fetchall():
    print(f"- {i[1]} {i[2]} was recommended by {i[4]} {i[5]}")

- Anna Mackenzie was recommended by Darren Smith
- Anne Baker was recommended by Ponder Stibbons
- Charles Owen was recommended by Darren Smith
- David Jones was recommended by Janice Joplette
- David Pinker was recommended by Jemima Farrell
- Douglas Jones was recommended by David Jones
- Erica Crumpet was recommended by Tracy Smith
- Florence Bader was recommended by Ponder Stibbons
- Gerald Butters was recommended by Darren Smith
- Henrietta Rumney was recommended by Matthew Genting
- Henry Worthington-Smyth was recommended by Tracy Smith
- Jack Smith was recommended by Darren Smith
- Janice Joplette was recommended by Darren Smith
- Joan Coplin was recommended by Timothy Baker
- John Hunt was recommended by Millicent Purview
- Matthew Genting was recommended by Gerald Butters
- Millicent Purview was recommended by Tracy Smith
- Nancy Dare was recommended by Janice Joplette
- Ponder Stibbons was recommended by Burton Tracy
- Ramnaresh Sarwin was recommended by Florence Bader
- Tim B

## Q12

In [296]:
rs = con.execute('''SELECT b.facid, f.name facility_name, m.memid,(m.surname||' '||m.firstname) full_name,
COUNT(1) usage
FROM Members m JOIN Bookings b on m.memid = b.memid
JOIN Facilities f ON b.facid = f.facid WHERE m.memid <> 0
GROUP BY b.facid, m.memid
ORDER BY facility_name,full_name''')

pd.DataFrame(rs.fetchall())

,facid,facility_name,memid,full_name,usage
0,2,Badminton Court,15,Bader Florence,9
1,2,Badminton Court,12,Baker Anne,10
2,2,Badminton Court,16,Baker Timothy,7
3,2,Badminton Court,8,Boothe Tim,12
4,2,Badminton Court,5,Butters Gerald,20
...,...,...,...,...,...
197,1,Tennis Court 2,1,Smith Darren,19
198,1,Tennis Court 2,14,Smith Jack,1
199,1,Tennis Court 2,2,Smith Tracy,2
200,1,Tennis Court 2,9,Stibbons Ponder,31


## Q13

In [297]:
rs = con.execute('''SELECT f.name facility_name, strftime('%m',b.starttime) month, COUNT(1) usage
FROM Facilities f JOIN Bookings b ON f.facid = b.facid
WHERE b.memid <> 0
GROUP BY facility_name,month
ORDER BY facility_name,month''')

pd.DataFrame(rs.fetchall())

,facility_name,month,usage
0,Badminton Court,07,51
1,Badminton Court,08,132
2,Badminton Court,09,161
3,Massage Room 1,07,77
4,Massage Room 1,08,153
5,Massage Room 1,09,191
6,Massage Room 2,07,4
7,Massage Room 2,08,9
8,Massage Room 2,09,14
9,Pool Table,07,103
